In [3]:
import pandas as pd
import numpy as np
data=[["<=30","high","no","fair","no"],
      ["<=30","high","no","excellent","no"],
       ["31...40","high","no","fair","yes"],
       [">40","medium","no","fair","yes"],
      [">40","low","yes","fair","yes"],
      [">40","low","yes","excellent","no"],
      ["31...40","low","yes","excellent","yes"],
      ["<=30","medium","no","fair","no"],
       ["<=30","low","yes","fair","yes"],
       [">40","medium","yes","fair","yes"],
      ["<=30","medium","yes","excellent","yes"],
      ["31...40","medium","no","excellent","yes"],
       ["31...40","high","yes","fair","yes"],
      [">40","medium","no","excellent","no"]   
     ]
columnname=["age","income","student","credit_rating","buys_computer"]
df=pd.DataFrame(data,columns=columnname)
df

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31...40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31...40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [4]:
# Count occurrences of each class in 'buys_computer' column
class_counts = df['buys_computer'].value_counts()

# Calculate the prior probability for each class
total_instances = len(df)
prior_probabilities = class_counts / total_instances

print(prior_probabilities)

buys_computer
yes    0.642857
no     0.357143
Name: count, dtype: float64


In [5]:
columnname = ["age", "income", "student", "credit_rating", "buys_computer"]

df = pd.DataFrame(data, columns=columnname)

# Calculate class conditional densities for each feature and class
for column in df.columns[:-1]:  # Exclude the last column which is the target variable
    print(f"Class conditional densities for feature '{column}':")
    for value in df[column].unique():
        print(f"Value: {value}")
        for class_value in df['buys_computer'].unique():
            subset = df[df['buys_computer'] == class_value]
            count = subset[subset[column] == value].shape[0]
            total_count = subset.shape[0]
            density = count / total_count if total_count > 0 else 0
            print(f"Class '{class_value}' Density: {density}")

Class conditional densities for feature 'age':
Value: <=30
Class 'no' Density: 0.6
Class 'yes' Density: 0.2222222222222222
Value: 31...40
Class 'no' Density: 0.0
Class 'yes' Density: 0.4444444444444444
Value: >40
Class 'no' Density: 0.4
Class 'yes' Density: 0.3333333333333333
Class conditional densities for feature 'income':
Value: high
Class 'no' Density: 0.4
Class 'yes' Density: 0.2222222222222222
Value: medium
Class 'no' Density: 0.4
Class 'yes' Density: 0.4444444444444444
Value: low
Class 'no' Density: 0.2
Class 'yes' Density: 0.3333333333333333
Class conditional densities for feature 'student':
Value: no
Class 'no' Density: 0.8
Class 'yes' Density: 0.3333333333333333
Value: yes
Class 'no' Density: 0.2
Class 'yes' Density: 0.6666666666666666
Class conditional densities for feature 'credit_rating':
Value: fair
Class 'no' Density: 0.4
Class 'yes' Density: 0.6666666666666666
Value: excellent
Class 'no' Density: 0.6
Class 'yes' Density: 0.3333333333333333


In [6]:
from scipy.stats import chi2_contingency
import pandas as pd


# Selecting only the four specified features
selected_features = ["age", "income", "student", "credit_rating"]
selected_df = df[selected_features]

# Perform chi-squared test for independence
chi2, p, dof, expected = chi2_contingency(pd.crosstab(selected_df['age'], [selected_df['income'], selected_df['student'], selected_df['credit_rating']]))

print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p}")


Chi-squared statistic: 12.95
P-value: 0.6764100579553458


In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

# Encoding categorical variables
encoder = LabelEncoder()
for col in df.columns:
    df[col] = encoder.fit_transform(df[col])

# Separating features and target variable
features = df.drop('buys_computer', axis=1)
target = df['buys_computer']

# Initialize Gaussian Naive Bayes model
model = GaussianNB()

# Fit the model with training data
model.fit(features, target)


GaussianNB()

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report


# Separating features and target variable
X = df.drop('buys_computer', axis=1)
y = df['buys_computer']

# Encoding categorical variables if necessary
X = pd.get_dummies(X)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Gaussian Naive Bayes model
model = GaussianNB()

# Fit the model with training data
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

report = classification_report(y_test, predictions)
print("Classification Report:")
print(report)



Accuracy: 0.3333333333333333
Classification Report:
              precision    recall  f1-score   support

          no       0.00      0.00      0.00         1
         yes       0.50      0.50      0.50         2

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3



In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing


data=pd.read_csv("SolarPrediction.csv")

# Selecting relevant columns for features and target
features = data[['Temperature', 'Pressure', 'Humidity', 'WindDirection(Degrees)', 'Speed']]
target = data['Radiation']

quartiles = target.quantile([0.25, 0.5, 0.75])

# Define thresholds based on quartiles
low_threshold = quartiles[0.25]
high_threshold = quartiles[0.75]

# Function to categorize radiation levels
def categorize_radiation(radiation):
    if radiation <= low_threshold:
        return 'Low'
    elif radiation <= high_threshold:
        return 'Moderate'
    else:
        return 'High'

# Apply categorization function to create labels
data['Radiation_Category'] = target.apply(categorize_radiation)

# Separate features and newly created target categories
features = data[['Temperature', 'Pressure', 'Humidity', 'WindDirection(Degrees)', 'Speed']]
target_categories = data['Radiation_Category']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_categories, test_size=0.2, random_state=42)


# Initialize Gaussian Naive Bayes model
model = GaussianNB()

# Fit the model with training data
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)



In [25]:
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

report = classification_report(y_test, predictions)
print("Classification Report:")
print(report)

Accuracy: 0.6394921994493729
Classification Report:
              precision    recall  f1-score   support

        High       0.74      0.74      0.74      1570
         Low       0.58      0.69      0.63      1946
    Moderate       0.64      0.55      0.59      3022

    accuracy                           0.64      6538
   macro avg       0.65      0.66      0.65      6538
weighted avg       0.64      0.64      0.64      6538

